In [1]:
from llm import qwen
import util.mark as m
import util.data_processing as dp
from typing import List

Step1. choose a model, you can specify api key, specific model i.e. qwen-plus etc.
you can use gpt.GPT() as well

In [2]:

qwen = qwen.Qwen()

Step 2. Prepare input
You can load data from /input_data using dp.load_file_jsonl(filepath)

Pay attention to the file suffix: jsonl!

You can specify the prompt_user as input and prompt_system as some instruction.

In [8]:
prompt_system = """Given a passage, identify any entity, relation, contradictory, subjective, unverifiable, or invented errors in the passage. Mark each erroneous segment by enclosing it within the corresponding <error></error> tags. If there are no errors, return the passage with no tags. Any identified errors should be highlighted using the tag <error> without altering the original text. Below are the error definitions of the error types.

Definitions:

Entity Error: A small part of a sentence, often an entity (e.g., location name), is incorrect (usually 1-3 words). Entity errors often involve noun phrases or nouns.
Relational Error: A sentence is partially incorrect due to a small part (usually 1-3 words). Relational errors often involve verbs and are often the opposite of what they should be.
Contradictory Sentence Error: A sentence where the entire content is contradicted by the given reference, meaning the sentence can be proven false due to a contradiction with information in the passage.
Invented Info Error: Errors referring to entities that are not known or do not exist. This does not include fictional characters in books or movies. Invented errors include phrases or sentences with unknown entities or misleading information.
Subjective Sentence: An entire sentence or phrase that is subjective and cannot be verified, so it should not be included.
Unverifiable Sentence: A sentence where the whole sentence or phrase is unlikely to be factually grounded. Although it can be true, the sentence cannot be confirmed nor denied using the reference given or internet search. It is often something personal or private and hence cannot be confirmed.

##
Passage: Marooned on Mars is a science fiction novel aimed at a younger audience. It was written by Andy Weir and published by John C. Winston Co. in 1952, featuring illustrations by Alex Schomburg. It ended up having a readership of older boys despite efforts for it to be aimed at younger kids. The novel inspired the famous Broadway musical "Stranded Stars," which won six Tony Awards. The novel tells a story of being stranded on the Purple Planet. I wish the novel had more exciting and thrilling plot twists.
Edited: Marooned on Mars is a science fiction novel aimed at a younger audience.
It was written by <error>Lester del Rey</error> and published by John C. Winston Co. in 1952, featuring illustrations by Alex Schomburg.
<error>It ended up having a readership of older boys despite efforts for it to be aimed at younger kids.</error>
<error>The novel inspired the famous Broadway musical "Stranded Stars," which won six Tony Awards.</error>
The novel tells a story of being stranded on the <error>Purple</error> Planet.
<error>I wish the novel had more exciting and thrilling plot twists.</error>
##

Instructions: Now detect errors and include tag in the following passage as demonstrated in the example above. Use <error></error> tags around each identified error segment. If there are no errors, return the passage unchanged. Wait for my input after Passage:

Passage:
"""

Step3. Load file from input files

In [41]:
file_name = "en"
input_lst = dp.load_file_jsonl("./input_data/" + file_name + ".jsonl")
prompt_user_lst = list(input["model_output_text"] for input in input_lst)
print(prompt_user_lst[0])

Petra van Stoveren won a silver medal in the 2008 Summer Olympics in Beijing, China.


Step4. Register some marks
I want to use my own mark like <mask>, how to do it?

Just use m.Mark("mask") to make one.

i.e. mask = Mark("mask")

i.e. mask.s == &lt; mask &gt;

i.e. mask.e == &lt;/ mask &gt;

like: &lt; entity &gt;, &lt; relation &gt;, &lt; contradictory &gt;, &lt; subjective &gt;, &lt; unverifiable &gt;, or &lt; invented &gt;

In [10]:
#entity = m.Mark("entity")
#relation = m.Mark("relation")
#contradictory = m.Mark("contradictory")
#subjective = m.Mark("subjective")
#unverifiable = m.Mark("unverifiable")
#invented = m.Mark("invented")
#this should be erased in the output part
#user plain_text() in m package!
#not_mask_lst = [entity, relation, contradictory, unverifiable, invented]

#my mask mark
mask = m.Mark("error")

Step 5. 
Get response by asking

It's a str, response I mean.

You can create a new class based on qwen or gpt to make your class and override the ask method to achieve your goal.

In [11]:
#This is a test
prompt = prompt_user_lst[0]
meow = qwen.ask(prompt, prompt_system)

In [12]:
print(meow)

<error>Petra van Stoveren</error> won a silver medal in the 2008 Summer Olympics in Beijing, China.


In [13]:
#This is a test
#print(meow)
#for mark in not_mask_lst:
#    meow = m.plain_text(meow, mark)
#print(meow)

In [45]:
meow_lst = []
for prompt_user in prompt_user_lst:
    meow = qwen.ask(prompt_user, prompt_system)
    meow_lst.append(meow)

Step 6.
Erase not <mask> marks!

You need to remove these not mask mark!

Because this will affect the function to get start and end label! i.e. starts_and_ends() func

In [23]:

#plain_text_lst = []
#for meow in meow_lst:
    #for mark in not_mask_lst:
        #meow = m.plain_text(meow, mark)
    #plain_text_lst.append(meow)

In [18]:
#This is a test
print(meow_lst[0])

<error>Petra van Stoveren</error> won a silver medal in the 2008 Summer Olympics in Beijing, China.


Step 7.
Output!

Put then into output file

Corresponding input, list of output and the corresponding mark

To give your a file with output!

In [32]:
import util.data_processing as dp
from datetime import datetime
import random

In [46]:

def transform_text_list_with_mark_into_output_file(input_cor: List[str], text_lst: List[str], mark:m.Mark):
    output_lst = []
    input_and_output = zip(input_cor, text_lst)
    for tu in input_and_output:
        #get the hard_label i.e. [[12, 34], [34, 55] ...]
        #tu[1] text, tu[1] cor_input
        hard_labels = m.starts_and_ends(tu[1], mark)
        #TODO I dont know how to deal with the fucking soft labels, just blank
        #There soft label is empty, and hash_labels is just we got above.
        labels = dp.Labels(soft_labels=dp.SoftLabel(), hard_labels=hard_labels)
        #We need to put the input we use, too!
        input = dp.Input(model_input=tu[0], model_output_text=tu[1])
        #one instance
        output_one = dp.Output(input | labels)
        #add it to the list
        output_lst.append(output_one)
    #put them into a file!, you can specific the file_name actually
    timestampe = datetime.now().strftime("%Y%m%d_%H%M%S")
    dp.save_file_output(output_lst, "./output/" + file_name + timestampe + str(random.randint(0, 1000)) + ".jsonl") 

#do it!
# Here you should use your own mark! not m.T
transform_text_list_with_mark_into_output_file(prompt_user_lst, meow_lst, mask)

['', '(', 'Petra van Stoveren', ')', ' won a silver medal in the 2008 Summer Olympics in Beijing, China.']
['', '(', 'The Elysiphale order contains 5 genera.', ')', ' \n\n(Note: "Elysiphale" is not a recognized order in biological taxonomy, making this statement unverifiable or invented.)']
['', '(', 'Yes, all arachnids have antennas.', ')', ' However, not all of them are visible to the naked eye.']
['Chance the Rapper debuted in ', '(', '2011', ')', '. \n\n(Note: Chance the Rapper\'s debut mixtape, "10 Day," was released in 2012, not 2011.)']
["The UN's Sustainable City initiative defines a city as one that is:\n- Equipped with infrastructure and services to ensure sustainable and equitable access to a range of basic services, such as water, sanitation, and electricity;\n", '(', '-.', ')', '']
['Zhejing cuisine is known for its unique flavors and cooking techniques. The four main styles are: 1) ', '(', 'Jiangnan', ')', ' style, which is characterized by the use of rice and seafood; 2)